In [5]:
import pandas as pd
from datatable import dt, f

TODO: Deixar aqui apenas os dados brutos

# Aquisição dos dados brutos

Primeiramente, conforme falado na seção anterior, unimos os microdados de todos os anos para ter um olhar menos fechado em um único ano.

In [9]:
def get_path_table_by_year(year):
  path_table = '/'.join([
    '../data/raw',
    'censo_{year}.zip',
    'Microdados do Censo da Educaç╞o Superior {year}',
    'dados',
    'MICRODADOS_CADASTRO_CURSOS_{year}.CSV'
  ]).format(year = year)
  return path_table

In [29]:
def aggregate_raw_data(from_year, to_year):
  for y in range(from_year, to_year + 1):
    path_table = get_path_table_by_year(y)
    dataset = dt.fread(path_table)
  return dataset

In [30]:
df_raw = aggregate_raw_data(2010, 2019)
display(df_raw)

,NU_ANO_CENSO,NO_REGIAO,CO_REGIAO,NO_UF,SG_UF,CO_UF,NO_MUNICIPIO,CO_MUNICIPIO,IN_CAPITAL,TP_ORGANIZACAO_ACADEMICA,…,QT_CONC_ATIV_EXTRACURRICULAR,QT_MOB_ACADEMICA,QT_ING_MOB_ACADEMICA,QT_MAT_MOB_ACADEMICA,QT_CONC_MOB_ACADEMICA
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪,▪▪▪▪,,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,2019,Centro-Oeste,5,Mato Grosso,MT,51,Água Boa,5100201,0,1,…,0,0,0,0,0
1,2019,Centro-Oeste,5,Mato Grosso,MT,51,Água Boa,5100201,0,1,…,0,0,0,0,0
2,2019,Centro-Oeste,5,Mato Grosso,MT,51,Água Boa,5100201,0,1,…,0,0,0,0,0
3,2019,Centro-Oeste,5,Mato Grosso,MT,51,Alto Araguaia,5100300,0,1,…,0,0,0,0,0
4,2019,Centro-Oeste,5,Mato Grosso,MT,51,Aripuanã,5101407,0,1,…,0,0,0,0,0
5,2019,Centro-Oeste,5,Mato Grosso,MT,51,Aripuanã,5101407,0,1,…,0,0,0,0,0
6,2019,Centro-Oeste,5,Mato Grosso,MT,51,Aripuanã,5101407,0,1,…,0,0,0,0,0
7,2019,Centro-Oeste,5,Mato Grosso,MT,51,Barra do Garças,5101803,0,1,…,4,0,0,0,0
8,2019,Centro-Oeste,5,Mato Grosso,MT,51,Barra do Garças,5101803,0,1,…,6,0,0,0,0


Para salvar esses dados agregados em um csv, é necessário fazer o seguinte:

In [31]:
df_raw.to_csv('../data/raw/aggregated_dataset.csv')

In [101]:
# Transformando datatable em dataframe do pandas para facilitar
df = df_raw.to_pandas()
df.shape

(246081, 196)

Após a agregação, a tabela bruta para trabalho ficou com 246,081 linhas e 196 colunas.  
Após isso, era necessário entender mais sobre as principais características e atributos da base que nos seriam interessantes.  
Para isso, é necessário conhecer a base primeiro, o que será feito nas células seguintes:

In [103]:
# Permitindo todas as colunas
pd.set_option("display.max.columns", None)
# Conhecendo o dataset a partir de alguns poucos registros
df.head(30)

,NU_ANO_CENSO,NO_REGIAO,CO_REGIAO,NO_UF,SG_UF,CO_UF,NO_MUNICIPIO,CO_MUNICIPIO,IN_CAPITAL,TP_ORGANIZACAO_ACADEMICA,TP_CATEGORIA_ADMINISTRATIVA,TP_REDE,CO_IES,NO_CINE_ROTULO,CO_CINE_ROTULO,CO_CINE_AREA_GERAL,NO_CINE_AREA_GERAL,CO_CINE_AREA_ESPECIFICA,NO_CINE_AREA_ESPECIFICA,CO_CINE_AREA_DETALHADA,NO_CINE_AREA_DETALHADA,TP_GRAU_ACADEMICO,TP_MODALIDADE_ENSINO,TP_NIVEL_ACADEMICO,QT_CURSO,QT_VG_TOTAL,QT_VG_TOTAL_DIURNO,QT_VG_TOTAL_NOTURNO,QT_VG_TOTAL_EAD,QT_VG_NOVA,QT_VG_PROC_SELETIVO,QT_VG_REMANESC,QT_VG_PROG_ESPECIAL,QT_INSCRITO_TOTAL,QT_INSCRITO_TOTAL_DIURNO,QT_INSCRITO_TOTAL_NOTURNO,QT_INSCRITO_TOTAL_EAD,QT_INSC_VG_NOVA,QT_INSC_PROC_SELETIVO,QT_INSC_VG_REMANESC,QT_INSC_VG_PROG_ESPECIAL,QT_ING,QT_ING_FEM,QT_ING_MASC,QT_ING_DIURNO,QT_ING_NOTURNO,QT_ING_VG_NOVA,QT_ING_VESTIBULAR,QT_ING_ENEM,QT_ING_AVALIACAO_SERIADA,QT_ING_SELECAO_SIMPLIFICA,QT_ING_EGR,QT_ING_OUTRO_TIPO_SELECAO,QT_ING_PROC_SELETIVO,QT_ING_VG_REMANESC,QT_ING_VG_PROG_ESPECIAL,QT_ING_OUTRA_FORMA,QT_ING_0_17,QT_ING_18_24,QT_ING_25_29,QT_ING_30_34,QT_ING_35_39,QT_ING_40_49,QT_ING_50_59,QT_ING_60_MAIS,QT_ING_BRANCA,QT_ING_PRETA,QT_ING_PARDA,QT_ING_AMARELA,QT_ING_INDIGENA,QT_ING_CORND,QT_MAT,QT_MAT_FEM,QT_MAT_MASC,QT_MAT_DIURNO,QT_MAT_NOTURNO,QT_MAT_0_17,QT_MAT_18_24,QT_MAT_25_29,QT_MAT_30_34,QT_MAT_35_39,QT_MAT_40_49,QT_MAT_50_59,QT_MAT_60_MAIS,QT_MAT_BRANCA,QT_MAT_PRETA,QT_MAT_PARDA,QT_MAT_AMARELA,QT_MAT_INDIGENA,QT_MAT_CORND,QT_CONC,QT_CONC_FEM,QT_CONC_MASC,QT_CONC_DIURNO,QT_CONC_NOTURNO,QT_CONC_0_17,QT_CONC_18_24,QT_CONC_25_29,QT_CONC_30_34,QT_CONC_35_39,QT_CONC_40_49,QT_CONC_50_59,QT_CONC_60_MAIS,QT_CONC_BRANCA,QT_CONC_PRETA,QT_CONC_PARDA,QT_CONC_AMARELA,QT_CONC_INDIGENA,QT_CONC_CORND,QT_ING_NACBRAS,QT_ING_NACESTRANG,QT_MAT_NACBRAS,QT_MAT_NACESTRANG,QT_CONC_NACBRAS,QT_CONC_NACESTRANG,QT_ALUNO_DEFICIENTE,QT_ING_DEFICIENTE,QT_MAT_DEFICIENTE,QT_CONC_DEFICIENTE,QT_ING_FINANC,QT_ING_FINANC_REEMB,QT_ING_FIES,QT_ING_RPFIES,QT_ING_FINANC_REEMB_OUTROS,QT_ING_FINANC_NREEMB,QT_ING_PROUNII,QT_ING_PROUNIP,QT_ING_NRPFIES,QT_ING_FINANC_NREEMB_OUTROS,QT_MAT_FINANC,QT_MAT_FINANC_REEMB,QT_MAT_FIES,QT_MAT_RPFIES,QT_MAT_FINANC_REEMB_OUTROS,QT_MAT_FINANC_NREEMB,QT_MAT_PROUNII,QT_MAT_PROUNIP,QT_MAT_NRPFIES,QT_MAT_FINANC_NREEMB_OUTROS,QT_CONC_FINANC,QT_CONC_FINANC_REEMB,QT_CONC_FIES,QT_CONC_RPFIES,QT_CONC_FINANC_REEMB_OUTROS,QT_CONC_FINANC_NREEMB,QT_CONC_PROUNII,QT_CONC_PROUNIP,QT_CONC_NRPFIES,QT_CONC_FINANC_NREEMB_OUTROS,QT_ING_RESERVA_VAGA,QT_ING_RVREDEPUBLICA,QT_ING_RVETNICO,QT_ING_RVPDEF,QT_ING_RVSOCIAL_RF,QT_ING_RVOUTROS,QT_MAT_RESERVA_VAGA,QT_MAT_RVREDEPUBLICA,QT_MAT_RVETNICO,QT_MAT_RVPDEF,QT_MAT_RVSOCIAL_RF,QT_MAT_RVOUTROS,QT_CONC_RESERVA_VAGA,QT_CONC_RVREDEPUBLICA,QT_CONC_RVETNICO,QT_CONC_RVPDEF,QT_CONC_RVSOCIAL_RF,QT_CONC_RVOUTROS,QT_SIT_TRANCADA,QT_SIT_DESVINCULADO,QT_SIT_TRANSFERIDO,QT_SIT_FALECIDO,QT_ING_PROCESCPUBLICA,QT_ING_PROCESCPRIVADA,QT_ING_PROCNAOINFORMADA,QT_MAT_PROCESCPUBLICA,QT_MAT_PROCESCPRIVADA,QT_MAT_PROCNAOINFORMADA,QT_CONC_PROCESCPUBLICA,QT_CONC_PROCESCPRIVADA,QT_CONC_PROCNAOINFORMADA,QT_PARFOR,QT_ING_PARFOR,QT_MAT_PARFOR,QT_CONC_PARFOR,QT_APOIO_SOCIAL,QT_ING_APOIO_SOCIAL,QT_MAT_APOIO_SOCIAL,QT_CONC_APOIO_SOCIAL,QT_ATIV_EXTRACURRICULAR,QT_ING_ATIV_EXTRACURRICULAR,QT_MAT_ATIV_EXTRACURRICULAR,QT_CONC_ATIV_EXTRACURRICULAR,QT_MOB_ACADEMICA,QT_ING_MOB_ACADEMICA,QT_MAT_MOB_ACADEMICA,QT_CONC_MOB_ACADEMICA
0,2019,Centro-Oeste,5.0,Mato Grosso,MT,51.0,Água Boa,5.10e+06,False,1,1,1,1,Pedagogia,0113P01,1,Educação,11,Educação,113,Formação de professores sem áreas específicas,2.0,2,1,0,0,0,0,0,0,False,0,0,0,0,0,0,0,False,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,41,39,2,0,0,0,4,5,13,12,7,0,0,11,5,21,3,1,0,0,0,0,0,0,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,41,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,3,0,0,0,0,0,37,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2019,Centro-Oeste,5.0,Mato Grosso,MT,51.0,Água Boa,5.10e+06,False,1,1,1,1,Computação formação de professor,0114C05,1,Educação,11

In [106]:
# Conhecendo a base
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246081 entries, 0 to 246080
Columns: 196 entries, NU_ANO_CENSO to QT_CONC_MOB_ACADEMICA
dtypes: bool(5), float64(4), int32(177), object(10)
memory usage: 193.6+ MB


In [127]:
# Descobrindo as colunas
df.columns.values

array(['NU_ANO_CENSO', 'NO_REGIAO', 'CO_REGIAO', 'NO_UF', 'SG_UF',
       'CO_UF', 'NO_MUNICIPIO', 'CO_MUNICIPIO', 'IN_CAPITAL',
       'TP_ORGANIZACAO_ACADEMICA', 'TP_CATEGORIA_ADMINISTRATIVA',
       'TP_REDE', 'CO_IES', 'NO_CINE_ROTULO', 'CO_CINE_ROTULO',
       'CO_CINE_AREA_GERAL', 'NO_CINE_AREA_GERAL',
       'CO_CINE_AREA_ESPECIFICA', 'NO_CINE_AREA_ESPECIFICA',
       'CO_CINE_AREA_DETALHADA', 'NO_CINE_AREA_DETALHADA',
       'TP_GRAU_ACADEMICO', 'TP_MODALIDADE_ENSINO', 'TP_NIVEL_ACADEMICO',
       'QT_CURSO', 'QT_VG_TOTAL', 'QT_VG_TOTAL_DIURNO',
       'QT_VG_TOTAL_NOTURNO', 'QT_VG_TOTAL_EAD', 'QT_VG_NOVA',
       'QT_VG_PROC_SELETIVO', 'QT_VG_REMANESC', 'QT_VG_PROG_ESPECIAL',
       'QT_INSCRITO_TOTAL', 'QT_INSCRITO_TOTAL_DIURNO',
       'QT_INSCRITO_TOTAL_NOTURNO', 'QT_INSCRITO_TOTAL_EAD',
       'QT_INSC_VG_NOVA', 'QT_INSC_PROC_SELETIVO', 'QT_INSC_VG_REMANESC',
       'QT_INSC_VG_PROG_ESPECIAL', 'QT_ING', 'QT_ING_FEM', 'QT_ING_MASC',
       'QT_ING_DIURNO', 'QT_ING_NOTURNO',

In [84]:
# Descobrindo as Áreas de Conhecimento Específico
specific_areas = df.drop_duplicates(subset=['CO_CINE_AREA_ESPECIFICA','NO_CINE_AREA_ESPECIFICA'], ignore_index=True)
print("Área Específica de Conhecimento")
specific_areas[['CO_CINE_AREA_ESPECIFICA','NO_CINE_AREA_ESPECIFICA']]

Área Específica de Conhecimento


,CO_CINE_AREA_ESPECIFICA,NO_CINE_AREA_ESPECIFICA
0,11,Educação
1,41,Negócios e administração
2,32,Comunicação e informação
3,42,Direito
4,61,Computação e Tecnologias da Informação e Comun...
5,73,Arquitetura e construção
6,81,Agricultura
7,91,Saúde
8,21,Artes
9,22,Humanidades (exceto línguas)


In [80]:
# Descobrindo as Áreas de Conhecimento Geral
areas = df.drop_duplicates(subset=['CO_CINE_AREA_GERAL','NO_CINE_AREA_GERAL'], ignore_index=True)
print("Área Geral de Conhecimento")
areas[['CO_CINE_AREA_GERAL','NO_CINE_AREA_GERAL']]

Área Geral de Conhecimento


,CO_CINE_AREA_GERAL,NO_CINE_AREA_GERAL
0,1,Educação
1,4,"Negócios, administração e direito"
2,3,"Ciências sociais, comunicação e informação"
3,6,Computação e Tecnologias da Informação e Comun...
4,7,"Engenharia, produção e construção"
5,8,"Agricultura, silvicultura, pesca e veterinária"
6,9,Saúde e bem-estar
7,2,Artes e humanidades
8,5,"Ciências naturais, matemática e estatística"
9,0,Programas básicos
